# Applied Data Science Capstone
## Practicing the handling of geolocation data and geospatial analysis

This notebook would be available on Github under the following link (Github handle @mukulbiswas)

In [ ]:
import pandas as pd
import Numpy as np

print('Hello Capstone Project Course!')